In [1]:
!pip install -q emoji
!pip install -q datasets
!pip install -q evaluate
!pip install -q rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [2]:
import os
import re
import json
import emoji
import torch
import evaluate
import numpy as np
from tqdm.notebook import tqdm
from datasets import load_dataset, Dataset
from transformers import (
    GPT2TokenizerFast,
    GPT2LMHeadModel,
    GPT2Config,
    EarlyStoppingCallback,
    TrainerCallback,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the codeparrot/apps from HuggingFace

In [3]:
dataset = load_dataset('codeparrot/apps')
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.63k [00:00<?, ?B/s]

apps.py:   0%|          | 0.00/4.95k [00:00<?, ?B/s]

The repository for codeparrot/apps contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/codeparrot/apps.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train.jsonl:   0%|          | 0.00/107M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/1.29G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['problem_id', 'question', 'solutions', 'input_output', 'difficulty', 'url', 'starter_code'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['problem_id', 'question', 'solutions', 'input_output', 'difficulty', 'url', 'starter_code'],
        num_rows: 5000
    })
})

In [ ]:
sample_idx = 0  # Change this index to view different samples
print(f"Problem ID: {dataset['train'][sample_idx]['problem_id']}")
print(f"Difficulty: {dataset['train'][sample_idx]['difficulty']}")
print(f"Problem Statement:\n{dataset['train'][sample_idx]['question']}")
print(f"Solution:\n{dataset['train'][sample_idx]['solutions']}")

Problem ID: 0
Difficulty: interview
Problem Statement:
Polycarp has $n$ different binary words. A word called binary if it contains only characters '0' and '1'. For example, these words are binary: "0001", "11", "0" and "0011100".

Polycarp wants to offer his set of $n$ binary words to play a game "words". In this game, players name words and each next word (starting from the second) must start with the last character of the previous word. The first word can be any. For example, these sequence of words can be named during the game: "0101", "1", "10", "00", "00001".

Word reversal is the operation of reversing the order of the characters. For example, the word "0111" after the reversal becomes "1110", the word "11010" after the reversal becomes "01011".

Probably, Polycarp has such a set of words that there is no way to put them in the order correspondent to the game rules. In this situation, he wants to reverse some words from his set so that:  the final set of $n$ words still contains

In [4]:
# Check the number of test cases per problem
test_case_counts = [len(item['input_output']) for item in tqdm(dataset['train']) if 'input_output' in item]
print(f"Min number of test cases: {min(test_case_counts) if test_case_counts else 'N/A'}")
print(f"Max number of test cases: {max(test_case_counts) if test_case_counts else 'N/A'}")
print(f"Average number of test cases: {np.mean(test_case_counts) if test_case_counts else 'N/A'}")

  0%|          | 0/5000 [00:00<?, ?it/s]

Min number of test cases: 0
Max number of test cases: 23613166
Average number of test cases: 5749.3512


In [5]:
no_test_cases = sum(1 for item in dataset['train'] if 'input_output' not in item or len(item['input_output']) == 0)
no_solutions = sum(1 for item in dataset['test'] if 'solutions' not in item or len(item['solutions']) == 0)
print(f'Problems without test cases in train split: {no_test_cases}')
print(f'Problems without solutions in test split: {no_solutions}')

Problems without test cases in train split: 195
Problems without solutions in test split: 1235


# Split and Clean the Data

In [6]:
def clean_question(question):
    question = re.sub(r'<[^>]*>', '', question) # Remove HTML
    question = emoji.replace_emoji(question, '') # Remove Emoji
    question = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()!@:%_\+.~#?&\/\/=]*)', '', question) # Remove URL
    question = re.sub(r'[^@ \t\r\n]+@[^@ \t\r\n]+\.[^@ \t\r\n]+', '', question) # Remove phone email
    question = re.sub(r'^[\+]?[(]?[0-9]{3}[)]?[-\s\.]?[0-9]{3}[-\s\.]?[0-9]{4,6}$', '', question) # Remove phone number
    question = re.sub(r'#\w+', '', question) # Remove hashtags
    question = re.sub(r"[^\sa-zA-Z0-9]", ' ', question) # Remove unnecessary characters
    question = re.sub(r'\n\n+s*\n', '\n', question) # Remove extra newline
    return re.sub(r'\s+', ' ', question).strip() # Remove extra whitespace

In [7]:
def clean_solution(solution):
    solution = re.sub(r'#\s*Time:.*|#\s*Space:.*|#\s*@author:.*|#\s*@date:.*', '', solution) # Remove comments that don't add value
    if 'def ' in solution or 'class ' in solution: # Skip imports, handle only function/class definitions
        # Try to find the first function or class definition
        first_def = solution.find('def ')
        first_class = solution.find('class ')

        # Find the earliest occurrence of either def or class
        start_idx = min(x for x in [first_def, first_class] if x >= 0) if first_def >= 0 or first_class >= 0 else 0
        solution = solution[start_idx:]

    # Remove trailing whitespace and ensure consistent newlines
    solution = '\n'.join(line.rstrip() for line in solution.strip().splitlines())
    return solution

In [8]:
# Split the train dataset into train and validation at the problem level to avoid leakage
# and create (question, solution) pairs, one per solution, for training and validation
train_val_split = dataset['train'].train_test_split(test_size=0.1, seed=42)
num_of_solutions = 1 # Number of solutions to take per question

train_data = [{'question': clean_question(sample['question']), 'solution': clean_solution(solution)}
    for sample in tqdm(train_val_split['train'])
    for solution in json.loads(sample['solutions'])[-num_of_solutions:]
]
val_data = [
    {'question': clean_question(sample['question']), 'solution': clean_solution(solution)}
    for sample in tqdm(train_val_split['test'])
    for solution in json.loads(sample['solutions'])[-num_of_solutions:]
]
test_data = [
    {'question': clean_question(sample['question']), 'solution': clean_solution(solution)}
    for sample in tqdm(dataset['test']) if sample['solutions']
    for solution in json.loads(sample['solutions'])[-num_of_solutions:]
]

  0%|          | 0/4500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

In [9]:
processed_data_path = 'processed_data'
os.makedirs(processed_data_path, exist_ok=True)

for split_name, split_data in zip(['train', 'val', 'test'], [train_data, val_data, test_data]):
    output_file = os.path.join(processed_data_path, f'{split_name}.json')
    with open(output_file, 'w') as f: # Save processed data splits to files
        json.dump(split_data, f, indent=2)
print(f'Extracted {len(train_data)} train, {len(val_data)} validation, and {len(test_data)} test examples')

Extracted 4500 train, 500 validation, and 3765 test examples


# Retrain the Tokenizer

In [10]:
# Extract questions and solutions from the train split to create a domain-specific corpus
questions = [sample['question'] for sample in dataset['train']]
solutions = [sol for sample in dataset['train'] for sol in json.loads(sample['solutions'])]

In [11]:
# Initialize a base tokenizer and train a new 1 on our corpus
base_tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
new_tokenizer = base_tokenizer.train_new_from_iterator(
    questions + solutions, # Combine natural language and code
    vocab_size = 50257,  # Match model's original vocab size for compatibility
    new_special_tokens = ['[CODE]']
)
new_tokenizer.pad_token = new_tokenizer.eos_token
new_tokenizer.save_pretrained('apps_tokenizer')  # Save the retrained tokenizer

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

('apps_tokenizer/tokenizer_config.json',
 'apps_tokenizer/special_tokens_map.json',
 'apps_tokenizer/vocab.json',
 'apps_tokenizer/merges.txt',
 'apps_tokenizer/added_tokens.json',
 'apps_tokenizer/tokenizer.json')

In [12]:
new_tokenizer = GPT2TokenizerFast.from_pretrained('apps_tokenizer')
test_input = 'def solve(nums):\n    return sum(nums)'
encoded = new_tokenizer.encode(test_input)
decoded = new_tokenizer.decode(encoded)
print(f'Testing tokenizer:\n'
      f'Original: {test_input}\n'
      f'Encoded: {encoded}\n'
      f'Decoded: {decoded}\n'
      f'Vocabulary size: {new_tokenizer.vocab_size}')

Testing tokenizer:
Original: def solve(nums):
    return sum(nums)
Encoded: [311, 1175, 9, 621, 283, 273, 296, 501, 9, 621, 10]
Decoded: def solve(nums):
    return sum(nums)
Vocabulary size: 50257


# Tokenization for Auto-regression Task

In [13]:
def tokenize_function(example): # tokenization function
    text = example['question'] + '\n[CODE]\n' + example['solution'] + new_tokenizer.eos_token
    inputs = new_tokenizer(text, truncation=True, padding='max_length', max_length=512)
    question = new_tokenizer(example['question'] + '\n[CODE]\n', truncation=True, max_length=512)

    # Create labels - we only want to compute loss on the solution part
    solution_start = len(question['input_ids'])  # Find where the solution starts in the encoded sequence
    labels = [-100] * solution_start + inputs['input_ids'][solution_start:] # Set labels to -100 for question part (ignored in loss calculation)
    inputs['labels'] = labels[:512]
    return inputs

In [14]:
train_dataset_processed = Dataset.from_list(train_data)
val_dataset_processed = Dataset.from_list(val_data)

tokenized_train_dataset = train_dataset_processed.map(tokenize_function, batched=False, remove_columns=train_dataset_processed.column_names)
tokenized_val_dataset = val_dataset_processed.map(tokenize_function, batched=False, remove_columns=val_dataset_processed.column_names)
tokenized_val_dataset # Dynamic padding will be handled by DataCollator

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 500
})

In [ ]:
# labels = np.array(tokenized_val_dataset[3]['labels'])
# labels = np.where(labels != -100, labels, new_tokenizer.pad_token_id)
# print(new_tokenizer.decode(labels))

# Metrics

In [15]:
bleu = evaluate.load('bleu')
rouge = evaluate.load('rouge')
meteor = evaluate.load('meteor')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [16]:
def preprocess_logits_for_metrics(logits, labels):
    '''
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    https://discuss.huggingface.co/t/cuda-out-of-memory-when-using-trainer-with-compute-metrics/2941/15
    '''
    pred_ids = torch.argmax(logits, dim=-1)
    return pred_ids, labels

In [17]:
def compute_metrics(eval_preds):
    preds = eval_preds.predictions[0]
    labels = eval_preds.label_ids
    preds = np.where(preds != -100, preds, new_tokenizer.pad_token_id) # Replace -100 with pad token id
    labels = np.where(labels != -100, labels, new_tokenizer.pad_token_id) # Replace -100 with pad token id

    # Decode predictions and labels
    decoded_preds = new_tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = new_tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute BLEU, ROUGE, and exact match score
    bleu_results = bleu.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    rouge_results = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    meteor_results = meteor.compute(predictions=decoded_preds, references=decoded_labels)
    # exact_match = sum(pred == label for pred, label in zip(decoded_preds, decoded_labels)) / len(decoded_preds)

    return {
        'bleu': bleu_results['bleu'],
        'rouge1': rouge_results['rouge1'],
        'rouge2': rouge_results['rouge2'],
        'rougeL': rouge_results['rougeL'],
        'meteor': meteor_results['meteor'],
        # 'exact_match': exact_match,
    }

In [18]:
class PerplexityCallback(TrainerCallback): # Define callback to compute perplexity from eval_loss
    def on_evaluate(self, args, state, control, metrics, **kwargs):
        if 'eval_loss' in metrics:
            perplexity = torch.exp(torch.tensor(metrics['eval_loss']))
            metrics['perplexity'] = perplexity.item()

# Training Setup

In [19]:
# config = GPT2Config.from_pretrained(
#     'gpt2',
#     vocab_size=new_tokenizer.vocab_size,
#     n_positions=1024,
#     # n_embd=768,
#     # n_layer=12,
#     # n_head=12
# )
# model = GPT2LMHeadModel(config) # Initialize a new model with this configuration
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(new_tokenizer))
total_params = sum(p.numel() for p in model.parameters())
print(f'Model initialized with {total_params / 1e6:.2f}M parameters')

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model initialized with 124.44M parameters


In [21]:
training_args = TrainingArguments(       # Define training arguments for fine-tuning
    output_dir='./results',              # Directory for checkpoints and logs
    num_train_epochs=20,                 #
    per_device_train_batch_size=32,      # Batch size per GPU
    per_device_eval_batch_size=32,       # Evaluation batch size
    learning_rate=2e-4,                  #
    # lr_scheduler_type='cosine',
    weight_decay=0.01,                   # Regularization
    logging_strategy='epoch',            #
    eval_strategy='epoch',               # Evaluate after each epoch
    save_strategy='epoch',               # Save after each epoch
    load_best_model_at_end=True,         # Load the best model based on validation loss
    metric_for_best_model='eval_loss',   # Use validation loss for early stopping
    greater_is_better=False,             # Lower loss is better
    fp16=torch.cuda.is_available(),      # Enable mixed-precision training if a CUDA GPU is available (faster, less memory)
)

# Fine-tune the Model

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer=new_tokenizer, mlm=False),
    processing_class=new_tokenizer,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics,
    callbacks=[PerplexityCallback, EarlyStoppingCallback(early_stopping_patience=3)]
)
trainer.train()  # Perform the fine-tuning
trainer.save_model('trained_model')  # Save the fine-tuned model

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 18520339 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor,
1,5.237800,4.477931,0.189259,0.515511,0.202136,0.348447,0.347182,88.052345
2,4.082500,4.052148,0.217689,0.559400,0.229816,0.379632,0.380652,57.520901
3,3.689000,3.845831,0.216021,0.570791,0.236493,0.387104,0.391533,46.797565
4,3.424300,3.708352,0.214552,0.587700,0.245158,0.399408,0.403954,40.786530
5,3.216000,3.628738,0.227819,0.587254,0.247642,0.403449,0.408902,37.665260
6,3.040700,3.568455,0.230690,0.591646,0.248027,0.405966,0.411826,35.461746
7,2.885700,3.525797,0.226111,0.591297,0.249809,0.408776,0.414168,33.980858
8,2.742100,3.508266,0.225062,0.598017,0.252330,0.415112,0.416566,33.390335
9,2.606300,3.524930,0.225645,0.598154,0.251840,0.415000,0.417760,33.951389
10,2.476700,3.528435,0.226266,0.597586,0.252531,0.415389,0.420319,34.070587


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


# Evaluation on Test Set

In [23]:
test_dataset_processed = Dataset.from_list(test_data)
tokenized_test_dataset = test_dataset_processed.map(tokenize_function, batched=False, remove_columns=test_dataset_processed.column_names)
trainer.evaluate(tokenized_test_dataset, metric_key_prefix='test')

Map:   0%|          | 0/3765 [00:00<?, ? examples/s]

early stopping required metric_for_best_model, but did not find eval_loss so early stopping is disabled


{'test_loss': 3.0332846641540527,
 'test_bleu': 0.2663857238343689,
 'test_rouge1': 0.6582246576883211,
 'test_rouge2': 0.30846048104612356,
 'test_rougeL': 0.4565441472586108,
 'test_meteor': 0.4530093454464635,
 'test_runtime': 343.2865,
 'test_samples_per_second': 10.968,
 'test_steps_per_second': 0.344,
 'epoch': 11.0}

# Inference

In [24]:
def generate_code(model, tokenizer, problem_text, device, max_length=512, num_beams=5, top_p=0.95, temperature=0.7):
    # Generate code solution for a given problem using beam search
    inputs = tokenizer(problem_text, return_tensors='pt', truncation=True, max_length=max_length)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    output = model.generate(
        **inputs,
        max_length=max_length,
        num_beams=num_beams,
        top_p=top_p,
        temperature=temperature,
        no_repeat_ngram_size=2,
        do_sample=True,
        early_stopping=True
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [25]:
question, solution = test_data[0]['question'], test_data[0]['solution']
pred_solution = generate_code(model, new_tokenizer, question, device)
print(question)
print(f'\n===== Grounth Truth: =====\n{solution}')
print(f'\n===== Prediction: =====\n{pred_solution}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


An accordion is a string yes in the real world accordions are musical instruments but let s forget about it for a while which can be represented as a concatenation of an opening bracket ASCII code 091 a colon ASCII code 058 some possibly zero vertical line characters ASCII code 124 another colon and a closing bracket ASCII code 093 The length of the accordion is the number of characters in it For example and are accordions having length 4 6 and 7 are not accordions You are given a string s You want to transform it into an accordion by removing some possibly zero characters from it Note that you may not insert new characters or reorder existing ones Is it possible to obtain an accordion by removing characters from s and if so what is the maximum possible length of the result Input The only line contains one string s 1 le s le 500000 It consists of lowercase Latin letters and characters and Output If it is not possible to obtain an accordion by removing some characters from s print 1 Oth

In [ ]:
# Preprocessing function
def preprocess_function(example):
    description = example["question"]
    code = example["solutions"][0]
    desc_tokens = tokenizer.tokenize("[DESC] " + description + " [CODE]")
    code_tokens = tokenizer.tokenize(code) + [tokenizer.eos_token]
    input_tokens = desc_tokens + code_tokens
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)

    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        code_start = len(desc_tokens)
        if code_start < MAX_LENGTH:
            labels = [-100] * code_start + input_ids[code_start:]
        else:
            labels = [-100] * MAX_LENGTH
    else:
        padding_length = MAX_LENGTH - len(input_ids)
        input_ids += [tokenizer.pad_token_id] * padding_length
        code_start = len(desc_tokens)
        labels = [-100] * code_start + input_ids[code_start:len(input_tokens)] + [-100] * padding_length

    return {"input_ids": input_ids, "labels": labels, "test_cases": example["input_output"]}

# Apply preprocessing
train_dataset = train_dataset.map(preprocess_function, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(preprocess_function, remove_columns=test_dataset.column_names)


# Train the model
trainer.train()
trainer.save_model(MODEL_DIR + "/final")